# Capstone Project part 2-1+2
### of IBM Applied Data Science on Coursera 
This notebook will be used to scrape a Wikipedia page for postal codes in Toronto, Canada and then putting into a pandas data frame.\
28-11-2019 Jonas Folesky

In [6]:
#imports
import pandas as pd

In [7]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [8]:
wikidata = pd.read_html(io='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

##### Loading the data with pd.read_html() does a fine job 

In [9]:
df_tn = wikidata[0] # dataframe toronto neighbourhoods
df_tn.rename(columns = {'Postcode':'Postal Code','Neighbourhood':'Neighborhood'},inplace=True) 
df_tn.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
##drop rows were Borough is not assigned
df_tn.drop(df_tn[df_tn['Borough']=='Not assigned'].index,inplace=True)

In [11]:
#join Neighborhoods with same postal code
df_tn_clean = df_tn.groupby('Postal Code').agg(Borough =('Borough','first'),
                                      Neighborhood = ('Neighborhood',', '.join)).reset_index() 
df_tn_clean.head()#check new dataframe

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
#replace not assigned neighborhood names with borough names
idx = df_tn_clean['Neighborhood']=='Not assigned'
df_tn_clean['Neighborhood'].loc[idx]=df_tn_clean['Borough'].loc[idx]
df_tn_clean.tail(10) #use n=10 because Queens Park (93) is the one row  affected 

,Postal Code,Borough,Neighborhood
93,M9A,Queen's Park,Queen's Park
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [13]:
df_tn_clean.shape

(103, 3)

#### Part 2 loading GeoData

In [14]:
# after trying geocoder package several times I decided to use the provided CSV file
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')

In [15]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_tn_full = df_tn_clean.join(coordinates.set_index('Postal Code'), on='Postal Code')

In [17]:
df_tn_full.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
#### I save the data frame to a csv file for lar

In [18]:
df_tn_full.to_csv('TorontoNeighborhoods.csv',index=False)